In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Veri ve klasör
df = pd.read_csv("birlesik_veri.csv", parse_dates=["datetime"])
df = df.sort_values("datetime")
os.makedirs("test_sets", exist_ok=True)

# Girdi-çıktı ayrımı
features = [col for col in df.columns if col not in ["datetime", "PTF (TL/MWh)"]]
target = "PTF (TL/MWh)"

# Modeli sadece eğit 10 günlük
train_df = df[df["datetime"] < "2024-07-01"]
test_dates = pd.date_range("2024-07-01", "2024-07-10", freq="D")

X_train = train_df[features]
y_train = train_df[target]

model = RandomForestRegressor().fit(X_train, y_train)

# Her tarih için test verisi al ve npy olarak kaydet
for dt in test_dates:
    test_df = df[(df["datetime"] >= dt) & (df["datetime"] < dt + pd.Timedelta(days=1))]
    if len(test_df) < 24: continue  # eksikse atla

    X_test = test_df[features]
    y_true = test_df[target].values
    y_pred = model.predict(X_test)

    date_str = dt.strftime("%Y%m%d")
    np.save(f"test_sets/y_true_{date_str}.npy", y_true)
    np.save(f"test_sets/y_pred_{date_str}.npy", y_pred)

print("Test setleri oluşturuldu.")


Test setleri oluşturuldu.


In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Test klasörü
test_dir = "test_sets"
os.makedirs("grafikler", exist_ok=True)

results = []

# Tüm dosyaları sırala
files = sorted(os.listdir(test_dir))

# Her tarih için X, y_true, y_pred yükle ve değerlendir
for file in files:
    if file.startswith("y_true_") and file.endswith(".npy"):
        date_str = file.split("_")[-1].split(".")[0]
        y_true_path = os.path.join(test_dir, f"y_true_{date_str}.npy")
        y_pred_path = os.path.join(test_dir, f"y_pred_{date_str}.npy")  # Bu dosya varsa kullanılacak

        if not os.path.exists(y_pred_path):
            continue  # Tahmin yoksa geç

        # Verileri yükle
        y_true = np.load(y_true_path)
        y_pred = np.load(y_pred_path)

        # Metrikler
        mae = mean_absolute_error(y_true, y_pred)
        mape = mean_absolute_percentage_error(y_true, y_pred) * 100

        results.append({
            "Tarih": date_str,
            "MAE": round(mae, 2),
            "MAPE (%)": round(mape, 2)
        })

        # Grafik çiz
        zaman = pd.date_range(start=pd.to_datetime(date_str), periods=len(y_true), freq="h")
        plt.figure(figsize=(12, 4))
        plt.plot(zaman, y_true, label="Gerçek PTF", color="blue")
        plt.plot(zaman, y_pred, label="Tahmin", color="orange", linestyle="--")
        plt.title(f"{date_str} Tahmini\nMAE={mae:.2f} TL/MWh, MAPE={mape:.2f}%")
        plt.xlabel("Saat")
        plt.ylabel("PTF (TL/MWh)")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"grafikler/tahmin_{date_str}.png")
        plt.close()

# Sonuçları tablo olarak göster
df_results = pd.DataFrame(results)
print(df_results)



C:\Users\Admin\AppData\Local\Temp\ipykernel_17160\1258239682.py:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  zaman = pd.date_range(start=pd.to_datetime(date_str), periods=len(y_true), freq="H")
C:\Users\Admin\AppData\Local\Temp\ipykernel_17160\1258239682.py:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  zaman = pd.date_range(start=pd.to_datetime(date_str), periods=len(y_true), freq="H")
C:\Users\Admin\AppData\Local\Temp\ipykernel_17160\1258239682.py:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  zaman = pd.date_range(start=pd.to_datetime(date_str), periods=len(y_true), freq="H")
C:\Users\Admin\AppData\Local\Temp\ipykernel_17160\1258239682.py:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  zaman = pd.date_range(start=pd.to_datetime(date_str), periods=len(y_true)

      Tarih     MAE  MAPE (%)
0  20240701  183.91      6.53
1  20240702  262.68      8.93
2  20240703  267.22      9.08
3  20240704  310.74     10.42
4  20240705  141.64      4.89
5  20240706   65.84      2.94
6  20240707   73.34      3.78
7  20240708  133.37      5.07
8  20240709  148.68      5.31
9  20240710  224.21      7.63
